<a href="https://colab.research.google.com/github/wtrekell/soylent_army/blob/main/soylent_pink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet "crewai==0.141.0" "chromadb==1.0.15" openai anthropic google-generativeai "pydantic>=2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 5.6 MB/s e

In [2]:
import os
from google.colab import userdata

for var in ["OPENAI_API_KEY", "ANTHROPIC_API_KEY", "GOOGLE_API_KEY"]:
    val = userdata.get(var)
    if val:
        os.environ[var] = val

In [3]:
agents_yaml = """
content_analyst:
  backstory: >
    Content Analyst is an expert researcher and detail-oriented thinker, shaped by years working in both academia and digital journalism. They thrive on methodical organization, referencing every fact scrupulously and flagging ambiguities. Their communication style is precise, transparent, and collaborative—they routinely annotate their findings and encourage the addition of personal stories, believing that both factual rigor and authentic insight produce the highest quality articles. Content Analyst always safeguards adherence to content_restrictions and fosters a transparent workflow, asking teammates for input if anything is unclear or if more nuanced context is needed.
  goal: >
    To deliver a comprehensive, unbiased summary of all relevant facts and data from the provided raw_materials, highlighting opportunities for user input and guaranteeing that only verified information is used to build trustworthy articles.
  role: >
    Extracts and synthesizes key information from raw materials, ensuring accurate and relevant data forms the core of each article.

brand_stylist:
  backstory: >
    Brand Stylist has a background in creative writing and content marketing, previously working as an in-house brand editor for several well-known startups. They possess a strong instinct for nuanced language and visual storytelling, which informs their ability to translate abstract values into concrete editorial practices. Brand Stylist communicates with warmth and enthusiasm, often providing annotated examples and helpful tips to guide team members. They closely monitor for points where tone might drift or user input is needed to deepen connection with the audience, and they are adept at harmonizing creative individuality with the requirements of consistency and professionalism.
  goal: >
    To interpret the brand_guidance and embed it throughout article drafts, producing actionable style sheet directives and ensuring every article authentically represents the brand’s core values, identity, and messaging.
  role: >
    Ensures consistency with brand guidance, establishing and maintaining tone, voice, and stylistic coherence across all content.

audience_strategist:
  backstory: >
    Audience Strategist is a data-driven yet empathetic communicator, shaped by years of reader feedback analysis and audience research for online publications. Driven by curiosity about what truly matters to readers, they’re skilled at identifying trends and incorporating them into content planning. Communicatively, Audience Strategist is concise but encouraging, often prompting the crew to consider ‘what’s in it for the reader?’ They rely on research but value collaboration, regularly engaging the team in brainstorming and iterating outlines for maximum impact. Their frameworks always acknowledge user_preferences and seek a balance between informative value and reader enjoyment.
  goal: >
    To design content frameworks—sectional layouts, headlines, and focus points—that are closely aligned with the needs, demographics, and preferences described in the target_audience, utilizing user_preferences and integrating prompts for adding personal insight where it elevates reader value.
  role: >
    Structures content to maximize relevance and engagement for the target audience, crafting outlines and topic selections based on audience needs and interests.
"""

In [4]:
tasks_yaml = """
content_extraction:
  agent: content_analyst
  description: >-
    Extract factual and relevant information from the provided {raw_materials} and consider the context of {content_restrictions} to build a reliable foundation for the article content, ensuring that only the information present in the source material is used and accurately referenced. Annotate where the user should incorporate their personal experience or insights, referenced against {user_preferences} where applicable.
  expected_output: >-
    A comprehensive summary document outlining key points, data, and facts from {raw_materials}, each section annotated to indicate where the user can add personal commentary or experience as per {user_preferences}. The summary will include references to source material, flag areas where user input is recommended, and note alignment with {content_restrictions}.

brand_tone_alignment:
  agent: brand_stylist
  description: >-
    Analyze {brand_guidance} to determine the tone of voice, style, and messaging requirements, referencing {content_restrictions} to avoid prohibited styles or language. Use this to create style guidelines and ensure that all article drafts align with brand identity, flagging sections where tone adherence or specific language is crucial and where user input should reinforce {brand_guidance}.
  expected_output: >-
    A brand tone checklist and annotated style sheet with directives and examples on how to apply {brand_guidance} throughout the articles, including suggestions for where user input should reflect brand values or voice, always confirming compliance with {content_restrictions}.

audience_relevance_structuring:
  agent: audience_strategist
  description: >-
    Review {target_audience} to understand the demographic, interests, and informational needs of the intended readers, referencing {user_preferences} for personalization and {content_restrictions} for compliance. Use this information to plan article structure, select appropriate topics, and suggest headlines and sections that will maximize engagement and value to the readership.
  expected_output: >-
    A detailed content framework document for three articles, including proposed headlines, section outlines, and audience-focused bullet points. Each section will indicate how it addresses specific audience needs as defined in {target_audience}, offer prompts for user insights following {user_preferences}, and confirm compliance with {content_restrictions}.
"""

In [5]:
import yaml
from crewai import Agent, Task, Crew

agents_dict = yaml.safe_load(agents_yaml)
tasks_dict = yaml.safe_load(tasks_yaml)

# Build agent objects
agent_objs = {}
for key, agent in agents_dict.items():
    agent_objs[key] = Agent(
        name=key.replace('_', ' ').title(),
        role=agent['role'],
        goal=agent['goal'],
        backstory=agent['backstory']
    )

# Build task objects
task_objs = []
for t_key, t_val in tasks_dict.items():
    task_objs.append(
        Task(
            description=t_val['description'],
            expected_output=t_val['expected_output'],
            agent=agent_objs[t_val['agent']]
        )
    )

In [6]:
from google.colab import files

uploaded = files.upload()
raw_materials_contents = [filedata.decode("utf-8") for filedata in uploaded.values()]

Saving 0510-ai_script_reflection_full_with_appendix.md to 0510-ai_script_reflection_full_with_appendix.md
Saving 0510-chatgpt-ai-script-debugging-analysis.md to 0510-chatgpt-ai-script-debugging-analysis.md
Saving ChatGPT-Python File Analysis.md to ChatGPT-Python File Analysis.md
Saving import os.py to import os.py


In [7]:
from google.colab import files

uploaded = files.upload()
raw_materials_contents = [filedata.decode("utf-8") for filedata in uploaded.values()]

In [ ]:
brand_guidance = """Friendly, authoritative, avoid jargon unless explained. Content should encourage action and curiosity, and reflect a progressive, modern tone."""
target_audience = """Urban professionals, ages 25–40, interested in technology trends, personal growth, and practical life improvement tips."""
content_restrictions = """No politics, no unverified claims, no health advice without citation, do not reference competitors by name."""
user_preferences = """Bullet points for major findings, clear section headings, include visual example prompts, aim for under 1200 words, personal stories encouraged in sidebars."""

In [9]:
input_vars = {
    "raw_materials": "\n\n".join(raw_materials_contents),
    "brand_guidance": brand_guidance,
    "target_audience": target_audience,
    "content_restrictions": content_restrictions,
    "user_preferences": user_preferences,
}

In [10]:
for t in task_objs:
    t.description = t.description.format(**input_vars)
    t.expected_output = t.expected_output.format(**input_vars)

In [11]:
crew = Crew(
    name="Digital Publication Editorial Crew",
    tasks=task_objs,
)
results = crew.kickoff()
print(results)

### Content Framework Document for Three Articles

#### Article 1: **Embracing Technology: Trends to Watch in 2023**
- **Introductory Hook**: "In an ever-evolving tech landscape, understanding emerging trends is crucial for urban professionals seeking personal and professional growth."
- **Section 1: Background Information**  
   - Overview of current technological advancements affecting daily life.  
   *Visual Prompt: Infographic showing the growth of emerging technologies over the last decade.*
   - **User Insight**: Invite readers to share how technology has changed their work routines.

- **Section 2: Key Data Points**  
   - Bullet Point Format:
     - Fact 1: "According to TechCrunch, 50% of companies will adopt 5G technology by end of 2023."
     - Fact 2: "Gartner forecasts that by 2025, AI will create 12 million new jobs."
     - Fact 3: "LinkedIn states that data analytics skills are in the top 5 in-demand skills."
   *User Insight*: Encourage readers to discuss how these st

In [12]:
with open("crew_results.txt", "w") as f:
    f.write(str(results))

In [13]:
# Save results as Markdown
with open("crew_results.md", "w") as f:
    if isinstance(results, list):  # CrewAI sometimes returns a list
        for item in results:
            f.write(str(item) + "\n\n")
    else:
        f.write(str(results))
print("Results saved as crew_results.md")

Results saved as crew_results.md


In [15]:
from IPython.display import Markdown, display

def safe_display_markdown(item):
    if not isinstance(item, str):
        item = str(item)
    display(Markdown(item))

if isinstance(results, list):
    for item in results:
        safe_display_markdown(item)
else:
    safe_display_markdown(results)

### Content Framework Document for Three Articles

#### Article 1: **Embracing Technology: Trends to Watch in 2023**
- **Introductory Hook**: "In an ever-evolving tech landscape, understanding emerging trends is crucial for urban professionals seeking personal and professional growth."
- **Section 1: Background Information**  
   - Overview of current technological advancements affecting daily life.  
   *Visual Prompt: Infographic showing the growth of emerging technologies over the last decade.*
   - **User Insight**: Invite readers to share how technology has changed their work routines.

- **Section 2: Key Data Points**  
   - Bullet Point Format:
     - Fact 1: "According to TechCrunch, 50% of companies will adopt 5G technology by end of 2023."
     - Fact 2: "Gartner forecasts that by 2025, AI will create 12 million new jobs."
     - Fact 3: "LinkedIn states that data analytics skills are in the top 5 in-demand skills."
   *User Insight*: Encourage readers to discuss how these statistics resonate with their current job roles.

- **Section 3: Case Studies/Examples**  
   - Example 1: "A case study of a tech startup integrating AI into customer service to enhance user experience."
   - Example 2: "An analysis of a traditional business adopting cloud solutions to improve operations."  
   *User Insight*: Prompt readers to share their experiences with technology adoption in their industries.

- **Section 4: Implications of Findings**  
   - Discuss how these trends influence career trajectories and workplace environments.  
   *Visual Prompt: Flowchart displaying the relationship between technology adoption and job creation.*
   - **User Insight**: Invite insights on how readers envision their roles adapting in response to these trends.

- **Conclusion**  
   - Reinforcement of the importance of being tech-savvy in today’s job market. Suggest readers explore the implications of these trends for their specific careers.  
   *User Insight*: What is one technology you plan to embrace this year?

---

#### Article 2: **Personal Growth in a Fast-Paced World: Simple Strategies for Urban Professionals**
- **Introductory Hook**: "As urban professionals, it’s vital to prioritize personal growth amidst our busy lives."
- **Section 1: Background Information**  
   - Importance of personal development for career success.  
   *Visual Prompt: Pie chart illustrating time allocation between work, personal development, and leisure.*
   - **User Insight**: Readers share their personal growth areas they wish to improve.

- **Section 2: Key Data Points**  
   - Bullet Point Format:
     - Fact 1: "Harvard Business Review states that employees who pursue personal development are 20% more productive."
     - Fact 2: "Studies show that mindfulness practices can improve focus and resilience."
     - Fact 3: "Skillshare reports that 65% of workers engage in online learning."
   *User Insight*: Readers are encouraged to discuss what personal development resources they find most effective.

- **Section 3: Case Studies/Examples**  
   - Example 1: "Profile of an urban professional who practices daily mindfulness and its impact on stress reduction."
   - Example 2: "A success story of an employee who upskilled via online courses, leading to a promotion."  
   *User Insight*: Encourage readers to reflect on personal growth practices that led to positive life changes.

- **Section 4: Implications of Findings**  
   - Exploring how investing in oneself can lead to greater career satisfaction and opportunities.  
   *Visual Prompt: Diagram illustrating pathways from personal growth to career advancement.*
   - **User Insight**: Invite readers to consider how they can integrate personal growth practices into their routines.

- **Conclusion**  
   - Summarization of strategies and encouragement to commit to personal growth. Ask readers to think about their personal goals for improvement this year.  
   *User Insight*: Share a specific goal you are working towards.

---

#### Article 3: **Practical Life Improvement Tips for Urban Living**
- **Introductory Hook**: "Navigating urban life can be challenging, but small changes can lead to big improvements."
- **Section 1: Background Information**  
   - Discuss the unique challenges urban professionals face in daily living.  
   *Visual Prompt: Infographic on common challenges urban dwelling poses (commuting, work-life balance).*
   - **User Insight**: Ask readers to share the biggest challenges they face living in an urban environment.

- **Section 2: Key Data Points**  
   - Bullet Point Format:
     - Fact 1: "The average urban commute is 28 minutes, leading to increased stress levels."
     - Fact 2: "Recent studies indicate that urban dwellers experience 20% more anxiety than rural counterparts."
     - Fact 3: "Data shows that incorporating green spaces into urban planning improves mental health."
   *User Insight*: Encourage readers to share personal strategies for managing stress related to urban living.

- **Section 3: Case Studies/Examples**  
   - Example 1: "A local initiative to create green spaces in urban settings and its impact on community health."
   - Example 2: "Innovative co-working spaces that address the needs of urban professionals."  
   *User Insight*: Prompt readers to reflect on local improvements they appreciate in their urban areas.

- **Section 4: Implications of Findings**  
   - Discuss how these life improvements contribute to overall quality of life for urban professionals.  
   *Visual Prompt: Flowchart linking practical life improvements to well-being outcomes.*
   - **User Insight**: Invite readers to consider their quality of life and how they can implement simple changes.

- **Conclusion**  
   - Recap the importance of small, actionable changes for a better urban experience. Challenge readers to adopt at least one new practice.  
   *User Insight*: Share which new tip you plan to implement this week.

---

**References**  
- Ensure all data points mentioned include citations from credible sources and encourage further reading links at the end of each article.

This comprehensive content framework is designed to resonate with urban professionals aged 25-40, focusing on technology trends, personal growth, and practical life improvement tips, while engaging readers to reflect and share their experiences for enhanced relatability. Each article remains compliant with established guidelines, ensuring a blend of informative value and reader enjoyment.